In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity", split="train")
print(dataset[0])

In [ ]:
import pandas as pd

# Convert only a small slice first (safe)
df = pd.DataFrame(dataset[:10])   # change 1000 → more later
df.head()

In [ ]:
df.head(15)

In [ ]:
# import csv

# with open("amazon_polarity_train.csv", "w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)
#     writer.writerow(["label", "title", "content"])

#     for item in dataset:
#         writer.writerow([item["label"], item["title"], item["content"]])

In [ ]:
import pandas as pd

df = pd.read_csv(
    "amazon_polarity_train.csv",
    engine="python",      # more tolerant parser
    on_bad_lines="skip"   # skip broken rows
)

df.head()

In [ ]:
print(len(df))

In [ ]:
df.tail()

In [ ]:
df.isnull().values.any()

In [ ]:
df.isnull().sum()

In [ ]:
(df.isnull().mean() * 100).round(2)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.info()

In [ ]:
df[df["title"].isna()]["label"].value_counts()

In [ ]:
df_filtered = df.dropna(subset=["title"])

In [ ]:
print("Original rows:", len(df))
print("Filtered rows:", len(df_filtered))
print("Rows removed:", len(df) - len(df_filtered))

In [ ]:
df_filtered.isnull().sum()

In [ ]:
df_filtered.to_csv(
    "ARC_Filtered.csv",
    index=False
)

In [ ]:
df = pd.read_csv("ARC_Filtered.csv")
len(df)

In [ ]:
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
df["text"] = df["title"] + " " + df["content"]

df["char_len"] = df["text"].str.len()
df["word_len"] = df["text"].str.split().str.len()

df[["char_len", "word_len"]].describe()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), ## Expansion
            GELU(), ## Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), ## Contraction
        )

    def forward(self, x):
        return self.layers(x)

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x
        # 2*4*768

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

import torch

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

In [ ]:
class AmazonPolarityDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, tokenizer, max_length=128):
        self.dataset = hf_dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item["title"] + " " + item["content"]

        token_ids = self.tokenizer.encode(text)
        token_ids = token_ids[:self.max_length]

        # padding
        if len(token_ids) < self.max_length:
            token_ids += [0] * (self.max_length - len(token_ids))

        return {
            "input_ids": torch.tensor(token_ids, dtype=torch.long),
            "labels": torch.tensor(item["label"], dtype=torch.long)
        }

In [ ]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
print(tokenizer.pad_token_id)

In [ ]:
from torch.utils.data import Subset, DataLoader

# take only 50k samples
subset_size = 100000
subset_indices = list(range(subset_size))

train_subset = Subset(dataset, subset_indices)

train_dataset = AmazonPolarityDataset(
    train_subset,
    tokenizer,
    max_length=128
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

In [ ]:
class GPTForClassification(nn.Module):
    def __init__(self, gpt, num_classes, pad_token_id):
        super().__init__()
        self.gpt = gpt
        self.pad_token_id = pad_token_id
        self.classifier = nn.Linear(
            gpt.out_head.in_features,  # emb_dim = 768
            num_classes
        )

    def forward(self, input_ids):
        seq_len = input_ids.size(1)

        # 🚀 GPT runs in inference mode
        with torch.no_grad():
            tok_embeds = self.gpt.tok_emb(input_ids)
            pos_embeds = self.gpt.pos_emb(
                torch.arange(seq_len, device=input_ids.device)
            )

            x = tok_embeds + pos_embeds
            x = self.gpt.drop_emb(x)
            x = self.gpt.trf_blocks(x)
            x = self.gpt.final_norm(x)

            # x = x[:, -1, :]   # last token pooling
            attention_mask = input_ids.ne(self.pad_token_id)   # BoolTensor
            last_token_idx = attention_mask.long().sum(dim=1) - 1

            x = x[torch.arange(x.size(0), device=x.device), last_token_idx]

        logits = self.classifier(x)
        return logits

In [ ]:
clf_model = GPTForClassification(
    model,
    num_classes=2,
    pad_token_id=tokenizer.pad_token_id
)

In [ ]:
for p in clf_model.gpt.parameters():
    p.requires_grad = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clf_model.to(device)

In [ ]:
batch = next(iter(train_loader))
print(type(batch))
print(batch)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
clf_model = clf_model.to(device)

In [ ]:
batch = next(iter(train_loader))

input_ids = batch["input_ids"].to(device)
labels = batch["labels"].to(device)

In [ ]:
%time logits = clf_model(input_ids)

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# clf_model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(
#     clf_model.classifier.parameters(),
#     lr=2e-4
# )

# clf_model.train()

# for epoch in range(3):
#     total_loss = 0

#     for step, batch in enumerate(train_loader):
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         logits = clf_model(input_ids)
#         loss = criterion(logits, labels)

#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#         if step % 10 == 0:
#             print(f"Epoch {epoch+1} | Step {step} | Loss {loss.item():.4f}")

#     print(f"Epoch {epoch+1} | Avg Loss {total_loss/len(train_loader):.4f}")

In [ ]:
class LoRALinear(nn.Module):
    def __init__(self, linear_layer, r=8, alpha=16):
        super().__init__()
        self.linear = linear_layer
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r

        in_dim = linear_layer.in_features
        out_dim = linear_layer.out_features

        self.lora_A = nn.Parameter(torch.randn(in_dim, r) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(r, out_dim))

        # freeze original weights
        for p in self.linear.parameters():
            p.requires_grad = False

    def forward(self, x):
        return self.linear(x) + (x @ self.lora_A @ self.lora_B) * self.scaling

In [ ]:
def apply_lora_to_gpt(gpt, r=8, alpha=16):
    for block in gpt.trf_blocks:
        block.att.W_query = LoRALinear(block.att.W_query, r, alpha)
        block.att.W_key   = LoRALinear(block.att.W_key, r, alpha)
        block.att.W_value = LoRALinear(block.att.W_value, r, alpha)

apply_lora_to_gpt(model, r=8, alpha=16)

In [ ]:
pad_token_id = tokenizer.pad_token_id

In [ ]:
print(pad_token_id)   # should be an integer

In [ ]:
num_classes = 2
clf_model = GPTForClassification(model, num_classes, pad_token_id)

In [ ]:
for p in clf_model.parameters():
    p.requires_grad = False

for name, p in clf_model.named_parameters():
    if "lora_" in name or "classifier" in name:
        p.requires_grad = True

In [ ]:
sum(p.numel() for p in clf_model.parameters() if p.requires_grad)

In [ ]:
print("Trainable parameters:")
for name, p in clf_model.named_parameters():
    if p.requires_grad:
        print(name, p.numel())

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clf_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, clf_model.parameters()),
    lr=3e-4
)

clf_model.train()

for epoch in range(2):   # keep short
    total_loss = 0
    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = clf_model(input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if step % 10 == 0:
            print(f"Epoch {epoch+1} | Step {step} | Loss {loss.item():.4f}")

    print(f"Epoch {epoch+1} | Avg Loss {total_loss/len(train_loader):.4f}")

In [ ]:
# Save model weights
torch.save(clf_model.state_dict(), "sentiment_model.pt")

# (Optional but recommended) Save tokenizer too
tokenizer.save_pretrained("tokenizer/")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

clf_model = GPTForClassification(
    gpt=model,                     # same GPT used during training
    num_classes=2,
    pad_token_id=tokenizer.pad_token_id
)

clf_model.load_state_dict(
    torch.load("sentiment_model.pt", map_location=device)
)

clf_model.to(device)
clf_model.eval()

# Load tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("tokenizer/")

In [ ]:
def predict(text):
    # clf_model.eval()

    enc = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    input_ids = enc["input_ids"].to(device)

    with torch.no_grad():
        logits = clf_model(input_ids)
        pred = torch.argmax(logits, dim=-1).item()

    return "Positive" if pred == 1 else "Negative"

In [ ]:
predict("This product is absolutely worst.")

In [ ]:
predict("This product is absolutely fantastic. I loved it.")

In [ ]:
predict("Worst purchase ever. Complete waste of money.")

In [ ]:
predict("Bad")

In [ ]:
predict("Cheap plastic, feels like it will snap.")

In [ ]:
predict("Doesn't do what the listing claims.")

In [ ]:
predict("Arrived crushed packaging.")

In [ ]:
predict("Arrived damaged packaging.")

In [ ]:
predict("Seller unresponsive to issues.")

In [ ]:
predict("Received the wrong size.")

In [ ]:
predict("Received the wrong item.")

In [ ]:
predict("Received the wrong color.")

In [ ]:
predict("Received the wrong item/color/size.")